In [ ]:
from google.cloud.bigquery.client import Client
import pandas as pd
from sklearn.model_selection import train_test_split

from src.config import ALL_COLUMNS, LABEL_COLUMN


def get_dataset_from_bq(project_id=None):
    client = Client(project=proyect_id)

    query_path = "../data/queries/flood_it_dataset.sql"
    with open(query_path, "r") as file:
        query = "".join(file.readlines())

    dataset = client.query(query).to_dataframe(create_bqstorage_client=False)
    
    return dataset

In [ ]:
output_path = "../data/users_raw.csv"
train_path = "../data/users_train.csv"
test_path = "../data/users_test.csv"
not_yet_path = "../data/users_not_yet.csv"

proyect_id = "project-name"
full_dataset = get_dataset_from_bq(proyect_id)

selector_train_test = (full_dataset["is_enable"] == 1) & (full_dataset["bounced"] == 0)
selectot_not_yet = (full_dataset["is_enable"] == 0) & (full_dataset["bounced"] == 0)

train, test = train_test_split(
    full_dataset.loc[selector_train_test],
    test_size=0.1,
    random_state=42,
    shuffle=True,
    stratify=full_dataset.loc[selector_train_test, LABEL_COLUMN],
)


full_dataset.to_csv(output_path, index=False)
full_dataset.loc[selectot_not_yet, :].to_csv(not_yet_path, index=False)
train.to_csv(train_path, index=False)
test.to_csv(test_path, index=False)
